# Forward model training

In this notebook we'll train the forward models NEIMS and RASSP that we use for fenerating syntetic datasets. In order to get through this notebook you first need to have the **python environments** set up (find more in README or the previous notebook). You also need the **NIST library** in the form of a .msp file. 

In [1]:
import sys 
sys.path.append('..')

## NIST splitting
**TODO**

In [2]:
nist_train_path = "data/nist/train.msp"
nist_test_path = "data/nist/test.msp"
nist_valid_path = "data/nist/valid.msp"

## NEIMS


### Conversion to SDF
First we need to convert the NIST library to SDF format that is supported by the NEIMS codebase. 

In [3]:
# takes approximately 15 mins to run
from spectra_process_utils import msp2sdf

msp2sdf(nist_test_path)
msp2sdf(nist_valid_path)
msp2sdf(nist_train_path, separate_replicates=True)

/home/xhajek9/miniconda3/envs/BARTtrainH100/lib/python3.8/site-packages/rdkit/Chem/PandasTools.py:371: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[molCol] = frame[smilesCol].map(Chem.MolFromSmiles)


```bash
cd deep-molecular-massspec
TARGET_PATH_NAME=tmp/massspec_predictions

python make_train_test_split.py --main_sdf_name=../data/nist/train_main.sdf \
                                --replicates_sdf_name=../data/nist/train_replicates.sdf \
                                --output_master_dir=$TARGET_PATH_NAME/spectra_tf_records

python molecule_estimator.py --dataset_config_file=$TARGET_PATH_NAME/spectra_tf_records/query_replicates_val_predicted_replicates_val.json \
                             --train_steps=100000 \
                             --model_dir=$TARGET_PATH_NAME/models/output \
                             --hparams=make_spectra_plots=True,batch_size=100 \
                             --alsologtostderr

```

In [4]:
conda activate NEIMSpy3 && python make_train_test_split.py --main_sdf_name=../data/nist/train_main.sdf --replicates_sdf_name=../data/nist/train_replicates.sdf --output_master_dir=$TARGET_PATH_NAME/spectra_tf_records


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




## RASSP